# **Installing packages and importing libraries**

In [ ]:

!pip install -q pyomo
from pyomo.environ import *
import numpy as np
import pandas as pd
import sys
!apt-get install -y -qq coinor-cbc



# **Ex2: Part 1 to 8** - *Problem Fomulation, Model, and Solver*

`In this problem, we have n different types of factories thtat we want to set up at n different locations such that each location has one factory. To solve this problem, we move ahead as follows :`

---
`Let i = 1,2,....,n-1,n denote the different type of factories. `

`Let j = 1,2,....,n-1,n denote the different type of location.`

`Let`

\begin{array}{l} \\
\ \ x_{ij} \ =\begin{cases}
1 & ,\ if\ i^{th} \ factory\ is\ setup\ at\ j^{th} \ location\\
0 & ,\ otherwise
\end{cases}
\end{array}


 \begin{array}{l}
Let\ C\ be\ the\ n\times n\ cost\ matrix\ \ where\ C_{ij} \ represents\ the\ cost\ of\ setting\ up\ the\ i^{th} \ factory\ at\ j^{th} \ location.\ Then\ our\ mathematical\ model\ looks\ like:\\
\\
Min\ :\ \sum ^{n}_{i=1}\sum ^{n}_{j=1} C_{ij} .x_{ij}\\
st.\\
\\
\sum ^{n}_{j=1} x_{ij} \ =\ 1\ \ \ \ \ \ \ \ for\ every\ \ \ i=1,2....,n-1,n\ \\
\sum ^{n}_{i=1} x_{ij} \ =\ 1\ \ \ \ \ \ \ \ for\ every\ \ \ j=1,2....,n-1,n\\
\\
the\ first\ constraint\ ensures\ that\ each\ type\ of\ factory\ is\ setup\ only\ at\ one\ location\ 
and\ second\ constraint\ ensures\ that\ each\ location\ gets\ ony\ one\ factory.
\end{array}




In [ ]:
#Cost matrix
cost = np.loadtxt('lab5_ex2.txt',delimiter=',',dtype=float)

if (cost.shape[0] != cost.shape[1]):
  sys.exit('The cost matrix is not a square matrix.')
  

N = cost.shape[0]
index = np.arange(N)

model = ConcreteModel()

model.x = Var(index,index,domain=NonNegativeIntegers)
model.obj= Objective(expr = sum(model.x[i,j] * cost[i,j] for i in index for j in index),sense=minimize)
model.cons =ConstraintList()

for k in index:
  model.cons.add( sum(model.x[k,j] for j in index) == 1)
  
for k in index:
  model.cons.add( sum(model.x[j,k] for j in index) == 1)
  
  
cbc = SolverFactory('cbc')
result = cbc.solve(model)

print('The cost of setting up the facotories (Optimal solution is : )',model.obj())
print('\n')

print('The solution is :')
print('\n')

for i in index:
  for j in index:
    if model.x[i,j].value>0:
      print('x[',i,',',j,'] : ', model.x[i,j].value,'     i.e Factory type',i+1,' at the location',j+1)





The cost of setting up the facotories (Optimal solution is : ) 198.0


The solution is :


x[ 0 , 1 ] :  1.0      i.e Factory type 1  at the location 2
x[ 1 , 6 ] :  1.0      i.e Factory type 2  at the location 7
x[ 2 , 11 ] :  1.0      i.e Factory type 3  at the location 12
x[ 3 , 9 ] :  1.0      i.e Factory type 4  at the location 10
x[ 4 , 2 ] :  1.0      i.e Factory type 5  at the location 3
x[ 5 , 10 ] :  1.0      i.e Factory type 6  at the location 11
x[ 6 , 3 ] :  1.0      i.e Factory type 7  at the location 4
x[ 7 , 5 ] :  1.0      i.e Factory type 8  at the location 6
x[ 8 , 4 ] :  1.0      i.e Factory type 9  at the location 5
x[ 9 , 0 ] :  1.0      i.e Factory type 10  at the location 1
x[ 10 , 7 ] :  1.0      i.e Factory type 11  at the location 8
x[ 11 , 8 ] :  1.0      i.e Factory type 12  at the location 9


# **Ex2: Part 9,10** - *Changing the variables to continuous variable and solving again*

***Remarks Below***

In [28]:
for i in index:
  for j in index:
    model.x[i,j].domain = NonNegativeReals



result = cbc.solve(model)

print('The cost of setting up the facotories (Optimal solution is : )',model.obj())
print('\n')

print('The solution is :')
print('\n')

for i in index:
  for j in index:
    if model.x[i,j].value>0:
      print('x[',i,',',j,'] : ', model.x[i,j].value,'     i.e Factory type',i+1,' at the location',j+1)





The cost of setting up the facotories (Optimal solution is : ) 198.0


The solution is :


x[ 0 , 1 ] :  1.0      i.e Factory type 1  at the location 2
x[ 1 , 6 ] :  1.0      i.e Factory type 2  at the location 7
x[ 2 , 11 ] :  1.0      i.e Factory type 3  at the location 12
x[ 3 , 9 ] :  1.0      i.e Factory type 4  at the location 10
x[ 4 , 2 ] :  1.0      i.e Factory type 5  at the location 3
x[ 5 , 10 ] :  1.0      i.e Factory type 6  at the location 11
x[ 6 , 3 ] :  1.0      i.e Factory type 7  at the location 4
x[ 7 , 5 ] :  1.0      i.e Factory type 8  at the location 6
x[ 8 , 4 ] :  1.0      i.e Factory type 9  at the location 5
x[ 9 , 0 ] :  1.0      i.e Factory type 10  at the location 1
x[ 10 , 7 ] :  1.0      i.e Factory type 11  at the location 8
x[ 11 , 8 ] :  1.0      i.e Factory type 12  at the location 9


***Remarks :  *** 

Even after changing the domain of the variables to the Positive real number the solution does not change at all. It's exactly same as the above where the domain was positive integers. 


Reason : The reason for this is that the constraint doesnt let the variable take any value. Because the sum of the rows of variable, as well as the columns of the variables has to be equal to 1. 


# **Ex2: Part 11** - *Changing the cost to non integer value in contionous case and solving again*

In this case we have solved the model again with the new cost matrix named, cost_fractions which now contain non integer cost value as wee. Also we have kept the domain of the variable as Non negative reals.

We observe that the cost in this case has risen to 198.45 from 198 in the previous part but the solution remains same. The solution still has integer values. So changing the cost matrix to fractional value changes the optimal cost and not the solution. 

In [36]:
cost_fractions = cost.copy()
cost_fractions[0] = [19.25,12.45,18.33,19.14,22.22,21.9,17.34,20.05,16.88,15.66,21.27,24.49]



if (cost_fractions.shape[0] != cost.shape[1]):
  sys.exit('The cost matrix is not a square matrix.')
  

N = cost_fractions.shape[0]
index = np.arange(N)

model = ConcreteModel()

model.x = Var(index,index,domain=NonNegativeReals)
model.obj= Objective(expr = sum(model.x[i,j] * cost_fractions[i,j] for i in index for j in index),sense=minimize)
model.cons =ConstraintList()

for k in index:
  model.cons.add( sum(model.x[k,j] for j in index) == 1)
  
for k in index:
  model.cons.add( sum(model.x[j,k] for j in index) == 1)
  
  
cbc = SolverFactory('cbc')
result = cbc.solve(model)

print('The cost of setting up the facotories (Optimal solution is : )',model.obj())
print('\n')

print('The solution is :')
print('\n')

for i in index:
  for j in index:
    if model.x[i,j].value>0:
      print('x[',i,',',j,'] : ', model.x[i,j].value,'     i.e Factory type',i+1,' at the location',j+1)


The cost of setting up the facotories (Optimal solution is : ) 198.45


The solution is :


x[ 0 , 1 ] :  1.0      i.e Factory type 1  at the location 2
x[ 1 , 6 ] :  1.0      i.e Factory type 2  at the location 7
x[ 2 , 11 ] :  1.0      i.e Factory type 3  at the location 12
x[ 3 , 9 ] :  1.0      i.e Factory type 4  at the location 10
x[ 4 , 2 ] :  1.0      i.e Factory type 5  at the location 3
x[ 5 , 10 ] :  1.0      i.e Factory type 6  at the location 11
x[ 6 , 3 ] :  1.0      i.e Factory type 7  at the location 4
x[ 7 , 5 ] :  1.0      i.e Factory type 8  at the location 6
x[ 8 , 4 ] :  1.0      i.e Factory type 9  at the location 5
x[ 9 , 0 ] :  1.0      i.e Factory type 10  at the location 1
x[ 10 , 7 ] :  1.0      i.e Factory type 11  at the location 8
x[ 11 , 8 ] :  1.0      i.e Factory type 12  at the location 9


# **Ex2: Part 12** - *Changing the model under the following conditions*



1.   Facility 1 cannot be assigned to Location 4
2.   Facility 11 cannot be assigned to Location 3
3.   Facility 5 cannot be assigned to Location 9

We can get around this problem in two ways:



1.   By setting the value of corresponding variables to 0.
2.   By increasing the cost of the concerened facility and location to a very high value so that solver does not take this into account while finding the solution.

We will, in this question go ahead with the first option, i.e we will add few more constraints and set the variable to 0.

So in the above problem in part 1 we add the following variables, 


 \begin{array}{l}
x_{0,3} \ =0\\
x_{10,2} \ =0\\
x_{4,8} \ =0
\end{array}






In [37]:
if (cost.shape[0] != cost.shape[1]):
  sys.exit('The cost matrix is not a square matrix.')
  

N = cost.shape[0]
index = np.arange(N)

model = ConcreteModel()

model.x = Var(index,index,domain=NonNegativeIntegers)
model.obj= Objective(expr = sum(model.x[i,j] * cost[i,j] for i in index for j in index),sense=minimize)
model.cons =ConstraintList()

for k in index:
  model.cons.add( sum(model.x[k,j] for j in index) == 1)
  
for k in index:
  model.cons.add( sum(model.x[j,k] for j in index) == 1)

model.cons.add(model.x[0,3]==0)  
model.cons.add(model.x[10,2]==0) 
model.cons.add(model.x[4,8]==0)   
  
cbc = SolverFactory('cbc')
result = cbc.solve(model)

print('The cost of setting up the facotories (Optimal solution is : )',model.obj())
print('\n')

print('The solution is :')
print('\n')

for i in index:
  for j in index:
    if model.x[i,j].value>0:
      print('x[',i,',',j,'] : ', model.x[i,j].value,'     i.e Factory type',i+1,' at the location',j+1)




The cost of setting up the facotories (Optimal solution is : ) 198.0


The solution is :


x[ 0 , 1 ] :  1.0      i.e Factory type 1  at the location 2
x[ 1 , 6 ] :  1.0      i.e Factory type 2  at the location 7
x[ 2 , 11 ] :  1.0      i.e Factory type 3  at the location 12
x[ 3 , 9 ] :  1.0      i.e Factory type 4  at the location 10
x[ 4 , 2 ] :  1.0      i.e Factory type 5  at the location 3
x[ 5 , 10 ] :  1.0      i.e Factory type 6  at the location 11
x[ 6 , 3 ] :  1.0      i.e Factory type 7  at the location 4
x[ 7 , 5 ] :  1.0      i.e Factory type 8  at the location 6
x[ 8 , 4 ] :  1.0      i.e Factory type 9  at the location 5
x[ 9 , 0 ] :  1.0      i.e Factory type 10  at the location 1
x[ 10 , 7 ] :  1.0      i.e Factory type 11  at the location 8
x[ 11 , 8 ] :  1.0      i.e Factory type 12  at the location 9
